In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
import librosa
import librosa.display
import IPython.display as ipd 
import os
import csv

In [ ]:
def slice_dataframe(machine):
    df = pd.read_csv('anomaly_files.csv')
    df_machine_sounds_abnorm = df[(df.dB == f'6_dB_{machine}') & (df.state == 'abnormal')]
    df_machine_sounds_norm = df[(df.dB == f'6_dB_{machine}') & (df.state == 'normal')]
    
    return df_machine_sounds_abnorm, df_machine_sounds_norm

In [ ]:
def extract_feature(file_path, target):
    X, sample_rate = librosa.load(file_path)

    feature_dict = {}
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T,axis=0)
    for idx, item in enumerate(mfccs):
        feature_dict[f'mfccs{idx}'] = item
        
    delta_mfccs = librosa.feature.delta(mfccs)
    for idx, item in enumerate(delta_mfccs):
        feature_dict[f'delta_mfccs{idx}'] = item
        
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    for idx, item in enumerate(delta2_mfccs):
        feature_dict[f'delta2_mfccs{idx}'] = item

    feature_dict['target'] = target
    return feature_dict

In [ ]:
def get_features(df_path, target):
    features = []
    for row in df_path.file_path:
        features.append(extract_feature(row, target))
        print(f'Extracted features for {row}')
    df_features = pd.DataFrame(features)    
    return df_features    

In [ ]:
def store_features(df_norm, df_abnorm, features_file_name):
    df_features = pd.concat([df_norm, df_abnorm])
    df_features.to_csv(features_file_name)

In [ ]:
def retrieve_and_store_feature(machine, features_file_name):
    print('slicing dataframe')
    df_machine_sounds_abnorm, df_machine_sounds_norm = slice_dataframe(machine)
    
    print('getting features')
    df_norm = get_features(df_machine_sounds_norm, 1)
    df_abnorm = get_features(df_machine_sounds_abnorm, 0)
    
    print('storing features')
    store_features(df_norm, df_abnorm, features_file_name)

In [ ]:
retrieve_and_store_feature('valve', 'df_6dB_valve_fe_ta_imb.csv')

In [ ]:
retrieve_and_store_feature('slider', 'df_6dB_slider_fe_ta_imb.csv')

In [ ]:
retrieve_and_store_feature('fan', 'df_6dB_fan_fe_ta_imb.csv')

In [ ]:
retrieve_and_store_feature('pump', 'df_6dB_pump_fe_ta_imb.csv')